# Near-surface NO₂ Estimation Using Artificial Neural Network (ANN) and Satellite–Meteorology Data

**Author:** Vigneshkumar Balamurugan  
**Description:**  
This notebook presents an ANN-based machine learning framework for estimating
near-surface NO₂ concentrations using TROPOMI, meteorological, land-use,
and emission-related predictors.

**Highlights**
- Reproducible ML workflow
- Multiple validation strategies (random, temporal, spatial)
- SHAP-based interpretability


## Imports & Configuration

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

## Data Loading

In [ ]:
df = pd.read_csv("data_for_ML_NO2_24hr_0.1.txt", sep=",", header=None) 

df.columns = ["date", "lon", "lat", "DOW", "Season", "Tropomi NO2", "RD","WS", "WD", "T", "RH", "BLH", "SP", "T2m", "DUV", "E", "EVI", "NDVI", "insitu_no2"]
df_new = df.drop([], axis=1)
df_new.dropna(inplace=True)

x = df_new.iloc[:, 3:-1] # input features
y = df_new.iloc[:, -1] # target feature


## Hyper Parameter Tuning

In [ ]:
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.3)
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

cs_cv_hy_pa = GridSearchCV(MLPRegressor(max_iter=2000), {'hidden_layer_sizes': [(200,100,50,),(200,100,50,25,),(300,150,75,37,),(400,200,100,50,)]}, cv=2)

cs_cv_hy_pa.fit(x_train, y_train)
print(pd.DataFrame(cs_cv_hy_pa.cv_results_))
print(cs_cv_hy_pa.best_score_)
print(cs_cv_hy_pa.best_params_)

# Evaluation: 70/30 Spilit

In [ ]:
ann_reg = MLPRegressor(hidden_layer_sizes=(200,100,50,), activation= 'tanh', alpha = 0.04, learning_rate = 'adaptive', solver = 'sgd', max_iter=2000)
ann_reg.fit(x_train,y_train)

y_pred = ann_reg.predict(x_test)

m, b = np.polyfit(y_test, y_pred, 1)

fig_dims = (10, 8)
fig, ax = plt.subplots(figsize=fig_dims)

plt.scatter(y_test, y_pred, s=100, edgecolors='black')
plt.plot(y_test, m*y_test + b,color='red',linewidth=5)
plt.grid(True,alpha=0.3)
plt.plot([0, 250], [0, 250], 'r--',linewidth=5)
plt.xlabel('Ground-truth $NO_2$ (µg m$^{-3}$)', fontsize=30)
plt.ylabel('Predicted $NO_2$ (µg m$^{-3}$)', fontsize=30)
plt.xlim([0, 120])
plt.ylim([0, 120])
plt.xticks([0, 50, 100],fontsize=30)
plt.yticks([50, 100],fontsize=30)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print ("R^2: ", r2)
print ("RMSE: ", mse**(1/2))
print ("slope: ", m)

## SHAP Interpretation (Feature Interpretation)

In [ ]:
x_train_summary = shap.kmeans(x_train, 10)

explainer = shap.KernelExplainer(ann_reg.predict,x_train_summary)
shap_values = explainer.shap_values(x_test)

plt.grid(True,alpha=0.3)
shap.summary_plot(shap_values, x_train, plot_type="bar",feature_names=x.columns)

# Evaluation: Random 5-fold CV

In [ ]:

ann_reg = MLPRegressor(hidden_layer_sizes=(200,100,50,), activation= 'tanh', alpha = 0.04, learning_rate = 'adaptive', solver = 'sgd', max_iter=2000)

df_new_2 = df_new.sample(frac=1)
x = df_new_2.iloc[:, 3:-1]
y = df_new_2.iloc[:, -1]

rmse_blo_ran_sam_app = []
r2_blo_ran_sam_app = []

for i in range(0,5):
    x_test = x.iloc[round(i*0.2*x.shape[0]):round((i+1)*0.2*x.shape[0]), :]
    y_test = y.iloc[round(i*0.2*y.shape[0]):round((i+1)*0.2*y.shape[0])]
    
    x_train = x
    x_train = x_train.drop (x_train.index[range(round(i*0.2*x.shape[0]),round((i+1)*0.2*x.shape[0]))])
    y_train = y
    y_train = y_train.drop (y_train.index[range(round(i*0.2*y.shape[0]),round((i+1)*0.2*y.shape[0]))])
    
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    ann_reg.fit(x_train,y_train)
    y_pred = ann_reg.predict(x_test)
    
    rmse_cv = (mean_squared_error(y_test, y_pred))**(1/2) 
    r2_cv = r2_score(y_test, y_pred)
    rmse_blo_ran_sam_app.append(rmse_cv)
    r2_blo_ran_sam_app.append(r2_cv)

print("Random CV R²:", np.mean(r2_blo_ran_sam_app), "±", np.std(r2_blo_ran_sam_app))
print("Random CV RMSE:", np.mean(rmse_blo_ran_sam_app), "±", np.std(rmse_blo_ran_sam_app))

# Evaluation: Time-leave-out 5-fold CV

In [ ]:
ann_reg = MLPRegressor(hidden_layer_sizes=(200,100,50,), activation= 'tanh', alpha = 0.04, learning_rate = 'adaptive', solver = 'sgd', max_iter=2000)


x = df_new.iloc[:, 3:-1]
y = df_new.iloc[:, -1]

rmse_blo_temp_sam_app = []
r2_blo_temp_sam_app = []

for i in range(0,5):
    x_test = x.iloc[round(i*0.2*x.shape[0]):round((i+1)*0.2*x.shape[0]), :]
    y_test = y.iloc[round(i*0.2*y.shape[0]):round((i+1)*0.2*y.shape[0])]
    
    x_train = x
    x_train = x_train.drop (x_train.index[range(round(i*0.2*x.shape[0]),round((i+1)*0.2*x.shape[0]))])
    y_train = y
    y_train = y_train.drop (y_train.index[range(round(i*0.2*y.shape[0]),round((i+1)*0.2*y.shape[0]))])
    
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    ann_reg.fit(x_train,y_train)
    y_pred = ann_reg.predict(x_test)
    
    rmse_cv = (mean_squared_error(y_test, y_pred))**(1/2) 
    r2_cv = r2_score(y_test, y_pred)
    rmse_blo_temp_sam_app.append(rmse_cv)
    r2_blo_temp_sam_app.append(r2_cv)

print("Time-leave-out CV R²:", np.mean(r2_blo_temp_sam_app), "±", np.std(r2_blo_temp_sam_app))
print("Time-leave-out CV RMSE:", np.mean(rmse_blo_temp_sam_app), "±", np.std(rmse_blo_temp_sam_app))


# Evaluation: Location-leave-out 5-fold CV

In [ ]:
ann_reg = MLPRegressor(hidden_layer_sizes=(200,100,50,), activation= 'tanh', alpha = 0.04, learning_rate = 'adaptive', solver = 'sgd', max_iter=2000)

x = df_new.iloc[:, 3:-1]
y = df_new.iloc[:, -1]

rmse_blo_spa_sam_app = []
r2_blo_spa_sam_app = []

lat_min = 47
div = 1.7

for i in range(0,5):
    
    df_test = df_new.loc[(df_new['lat'] >= lat_min) & (df_new['lat'] <= lat_min+div)]
    x_test = df_test.iloc[:, 3:-1]
    y_test = df_test.iloc[:, -1]
    
    df_train = df_new.drop(df_new[(df_new['lat'] >= lat_min) & (df_new['lat'] <= lat_min+div)].index)
    x_train = df_train.iloc[:, 3:-1]
    y_train = df_train.iloc[:, -1]
    
    lat_min += div
    
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    ann_reg.fit(x_train,y_train)
    y_pred = ann_reg.predict(x_test)
    
    rmse_cv = (mean_squared_error(y_test, y_pred))**(1/2) 
    r2_cv = r2_score(y_test, y_pred)
    rmse_blo_spa_sam_app.append(rmse_cv)
    r2_blo_spa_sam_app.append(r2_cv)

print("Location-leave-out CV R²:", np.mean(r2_blo_spa_sam_app), "±", np.std(r2_blo_spa_sam_app))
print("Location-leave-out CV RMSE:", np.mean(rmse_blo_spa_sam_app), "±", np.std(rmse_blo_spa_sam_app))
